In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


def GBM(n_estimators, lr):
    
#     data=pd.read_csv('utube_data.csv')
    data=pd.read_csv('https://raw.githubusercontent.com/esmou2/source-code-to-bow/master/Data/Ankidroid_bow_features.csv')

#     X = data.loc[:, data.columns != 'loves troll 2']
    X = data.loc[:, data.columns != 'is_vulnerable']
    y= data['is_vulnerable']
#     print(y.shape)
#     print(X.shape)

#     y= data['loves troll 2']
    
    # encoding 
    le = preprocessing.LabelEncoder()
    y_t=le.fit_transform(y)
    
    #splitting dataset 80% and 20%
    Test_split=0.2

    x_train,x_test,y_train, y_test= train_test_split(X,y_t, test_size=Test_split, random_state=42)
#     print(y_train)
#     print(x_train.shape)
    

    count_one=0
    for i in range(len(y_train)):
        if y_train[i]==1:
            count_one=count_one+1
    
    total=len(y_train)
    count_zero=total-count_one
    odds=np.divide(count_one,count_zero)
    log_odds=np.log(odds)
    y_prob=np.divide(np.exp(log_odds),np.add(1, np.exp(log_odds)))
    probab=np.full(y_train.shape, y_prob)
    y_proba= probab.tolist()
#     print("initial proba", y_proba)
   

    for _ in range(n_estimators):
        residual=[]
        for i in range(len(y_train)):
            res=y_train[i]-y_proba[i]
            residual.append(res)
#         print("residual is ", residual)

        estimators=[]
        a=()

        for j in range(len(x_train)):
            tree=DecisionTreeRegressor(max_depth= 2)
            tree.fit(x_train, residual)
            y_pred=tree.predict(x_train)
            leaves_index=tree.apply(x_train)
            #converting into tuples
            a=a+((leaves_index[j],y_pred[j]),)
        a=list(a)
        #     print(list(a))

        y_prob_temp=[]
        for i in range(len(a)):   
            upper_part=0
            lower_part=0
            log_odds=np.log(odds)

            for j in range(0, len(a)):
                if a[i][0]==a[j][0]:
                    # print("upper part is", upper_part)
                    upper_part=upper_part+a[j][1]
    #             print('updated upper', upper_part)
                    lower_part=lower_part+(np.multiply(y_proba[i],(1-y_proba[i])))
    #             print(upper_part)
            output=np.divide(upper_part,lower_part)
#             print("output is", output)
            log_odds= np.add((log_odds), np.multiply(lr, output)) 
            probabaility=np.divide(np.exp(log_odds),np.add(1,np.exp(log_odds)))
    #         print("probabaility is ", probabaility)

            y_prob_temp.append(probabaility)
    #     print(y_prob_temp)
        y_proba=list(y_prob_temp)

#         print("updated prob", y_proba)
        print("\n")
#     print(X. iloc[2])
#     pred=tree.predict([[1,12, 20]])
    pred=tree.predict(x_test)
    pred_train= tree.predict(x_train)
#     print(pred)
#     change into original label
    pred_main_train= [1 if m>0 else 0 for m in pred_train]
    pred_main=[1 if v>0 else 0 for v in pred]
    
#     if pred>0:
#         pred=1
#     else:
#         pred=0
#     print(le.inverse_transform(pred))
#     print(pred_main)
    acc_score=accuracy_score(y_test, pred_main)
    print("for train data")
    print(confusion_matrix(y_train, pred_main_train))
    print( "for test data")
    print(confusion_matrix(y_test, pred_main ))
    print('\n')
    return acc_score


In [6]:
GBM_class=GBM( n_estimators=4, lr=0.1)
print(GBM_class)









for train data
[[ 76  11]
 [ 16 120]]
for test data
[[18  8]
 [ 2 28]]


0.8214285714285714
